<a href="https://colab.research.google.com/github/yangyeji2020311166/ML/blob/main/TF_IDF_(Vectorizer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-17\"

In [ ]:
f=open('/content/스마트폰 크롤링.txt','r',encoding='utf-8')
txt = f.readlines()
f.close()

txt2=[]          #중복 댓글 제거
for item in txt:
    if item not in txt2:
        txt2.append(item)


print(txt2)

In [ ]:
##대댓글 아이디 삭제하기
remove_id_list=[]
for i in range(len(txt2)):
  if '?@' in txt2[i]:
    cnt = 0
    for j in range(len(txt2[i])):
      if (txt2[i][j] == '?' or txt2[i][j] == ' '):
        if (txt2[i][j] == '?'):
          cnt+=1
      elif cnt>=2:
        break
    tmp = txt2[i].split(sep='?',maxsplit=cnt)
    remove_id_list.append(tmp[cnt])
  else:
    remove_id_list.append(txt2[i])


In [ ]:
from konlpy.tag import Okt
import konlpy
okt = konlpy.tag.Okt()

for i, document in enumerate(remove_id_list):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(document, stem=True): #어간 추출
        if word[1] in ['Noun']: #명사
            clean_words.append(word[0])
    remove_id_list[i] = clean_words

In [ ]:
#불용어 제거
f = open('/content/스탑워드 + 하다.txt', "rt", encoding="utf-8") 
lines = f.readlines()
stopwords = []
unique_Noun_words=[]
for line in lines:
    line = line.replace('\n', '')
    stopwords.append(line)
f.close()
print(stopwords)


unique_Noun_words=[]
for i in range(len(remove_id_list)):#이차원 배열 전체에서 for 문 돌리기
  tmp=[]
  for j in range(len(remove_id_list[i])):#배열 안에서 일차원 배열의 길이가 서로 다 다르니깐 그 길이만큼 돌리면서
    if remove_id_list[i][j] not in stopwords:#불용어 있는지 검사 없으면 tmp에 임시저장
      tmp.append(remove_id_list[i][j])
  unique_Noun_words.append(tmp)#tmp에 임시저장한 리스트를 배열 전체 하나가 끝날때마다 검사

In [ ]:
test = []
for doc in unique_Noun_words:
    for word in doc:
    	# 특수문자, 숫자로만 구성된 단어, 1글자 단어
        if word.isalnum() == False or word.isdigit() == True or len(word) == 1:
#             print("특수")
            continue
        test.append(word)

# 등장 단어리스트 추출
vocab = list(set(test))
vocab.sort()

In [ ]:
tfidfwords = []
for x in range(len(unique_Noun_words)):
  tfidfwords.append((',').join(unique_Noun_words[x]))
print(tfidfwords)

In [ ]:
count_vec = CountVectorizer()     #글자수 세기
document_term_matrix = count_vec.fit_transform(tfidfwords).toarray()
print(document_term_matrix)

In [ ]:
tfidf_trans = TfidfTransformer()    #글자수 matrix를 활용해서 tf-idf 구하는 방법(1)
tfidf_trans.fit_transform(document_term_matrix).toarray()

In [ ]:
tfidf_vec = TfidfVectorizer()     #곧바로 tf-idf 구하는 방법(2)
tfidf_vec.fit_transform(tfidfwords).toarray()

In [ ]:
tfidf_dict = tfidf_vec.get_feature_names()

In [ ]:
tf_ = pd.DataFrame(document_term_matrix, columns = tfidf_dict)
tf_.sum().sort_values(ascending = False)[:50]

In [ ]:
tfidf_ = pd.DataFrame(tfidf_trans.fit_transform(document_term_matrix).toarray(), columns = tfidf_dict)

In [ ]:
tfidf_.sum().sort_values(ascending = False)[:50]

In [ ]:
tfidf2_ = pd.DataFrame(tfidf_vec.fit_transform(tfidfwords).toarray(), columns = tfidf_dict)

In [ ]:
tfidf2_.sum().sort_values(ascending = False)[:50]

In [ ]:
tfidfsum = pd.DataFrame(tfidf2_.sum().sort_values(ascending = False)[:50])
tfidfsum.head(10)
weightlist = []
for x in range(len(tfidfsum)):
  weightlist.append(tfidfsum.index[x])
weightdata = tfidfsum.to_dict('index')
print(weightdata)

In [ ]:
weightdict = {}
for x in weightlist:
  weightdict[x] = weightdata[x][0]
print(weightdict)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

!apt-get update -qq
!apt-get install fonts-nanum* -qq

font = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'
wordcloud = WordCloud(font_path = font, background_color='white',colormap = "Accent_r", 
                      width=3000, height=2000).generate_from_frequencies(weightdict)

plt.imshow(wordcloud) 
plt.axis('off') 
plt.show()